In [1]:
import sagemaker

sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name

bucket = sagemaker_session.default_bucket()
prefix = "sagemaker/dataset"

role = "arn:aws:iam::235411143540:role/SageMakerRole"


sagemaker.config INFO - Not applying SDK defaults from location: /opt/homebrew/share/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/viktor/Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /opt/homebrew/share/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/viktor/Library/Application Support/sagemaker/config.yaml


In [5]:
# inputs = sagemaker_session.upload_data(path="saved_dataset", bucket=bucket, key_prefix=prefix)
# print("input spec (in this case, just an S3 path): {}".format(inputs))
inputs = "s3://sagemaker-eu-central-1-235411143540/saved_dataset"

In [16]:
!pygmentize mnist.py


import argparse
import json
import logging
import os
import sys

#import sagemaker_containers
import torch
import torch.distributed as dist
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
import torch.utils.data.distributed
from torchvision import datasets, transforms

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(sys.stdout))


# Based on https://github.com/pytorch/examples/blob/master/mnist/main.py
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 32

In [6]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
    entry_point="faster_rcnn.py",
    source_dir='src',        # Directory where your source code is located
    role=role,
    py_version="py38",
    framework_version="1.11.0",
    instance_count=2,
    instance_type="ml.m4.xlarge", # ml.c5.2xlarge
    hyperparameters={"epochs": 10, "backend": "gloo"},
)


sagemaker.config INFO - Not applying SDK defaults from location: /opt/homebrew/share/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/viktor/Library/Application Support/sagemaker/config.yaml


In [7]:
estimator.fit({"training": inputs})


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


Using provided s3_resource


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: pytorch-training-2023-10-29-18-45-08-711


2023-10-29 18:45:10 Starting - Starting the training job...
2023-10-29 18:45:34 Starting - Preparing the instances for training......
2023-10-29 18:46:38 Downloading - Downloading input data...............
2023-10-29 18:49:14 Training - Downloading the training image...
2023-10-29 18:49:34 Training - Training image download completed. Training in progress....
2023-10-29 18:50:01 Uploading - Uploading generated training model
2023-10-29 18:50:01 Failed - Training job failed
..

UnexpectedStatusException: Error for Training job pytorch-training-2023-10-29-18-45-08-711: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
ExitCode 1
ErrorMessage "TypeError: 'type' object is not subscriptable"
Command "/opt/conda/bin/python3.8 faster_rcnn.py --backend gloo --epochs 10", exit code: 1

In [32]:
predictor = estimator.deploy(initial_instance_count=1, instance_type="ml.m4.xlarge")


INFO:sagemaker:Repacking model artifact (s3://sagemaker-eu-central-1-235411143540/pytorch-training-2023-10-27-19-14-02-631/output/model.tar.gz), script artifact (s3://sagemaker-eu-central-1-235411143540/pytorch-training-2023-10-27-19-14-02-631/source/sourcedir.tar.gz), and dependencies ([]) into single tar.gz file located at s3://sagemaker-eu-central-1-235411143540/pytorch-training-2023-10-27-19-49-51-655/model.tar.gz. This may take some time depending on model size...
INFO:sagemaker:Creating model with name: pytorch-training-2023-10-27-19-49-51-655
INFO:sagemaker:Creating endpoint-config with name pytorch-training-2023-10-27-19-49-51-655
INFO:sagemaker:Creating endpoint with name pytorch-training-2023-10-27-19-49-51-655


-------!

In [34]:
import gzip
import numpy as np
import random
import os

data_dir = "data/MNIST/raw"
with gzip.open(os.path.join(data_dir, "t10k-images-idx3-ubyte.gz"), "rb") as f:
    images = np.frombuffer(f.read(), np.uint8, offset=16).reshape(-1, 28, 28).astype(np.float32)

mask = random.sample(range(len(images)), 16)  # randomly select some of the test images
mask = np.array(mask, dtype=np.int32)
data = images[mask]


In [35]:
response = predictor.predict(np.expand_dims(data, axis=1))
print("Raw prediction result:")
print(response)
print()

labeled_predictions = list(zip(range(10), response[0]))
print("Labeled predictions: ")
print(labeled_predictions)
print()

labeled_predictions.sort(key=lambda label_and_prob: 1.0 - label_and_prob[1])
print("Most likely answer: {}".format(labeled_predictions[0]))


Raw prediction result:
[[    0.         -1531.03552246  -775.55126953  -882.36639404
  -1270.45324707  -764.815979    -803.70916748 -1069.17114258
   -846.71520996 -1144.79272461]
 [ -355.86022949  -440.81268311  -202.45556641  -266.35440063
   -401.78820801  -287.02746582  -264.79299927  -576.97924805
      0.          -450.95666504]
 [ -800.54821777  -702.08032227  -719.37127686  -437.40423584
    -19.32922363  -343.06335449  -536.19616699  -357.53234863
   -276.92120361     0.        ]
 [ -645.06445312  -817.94506836  -521.99169922  -694.60449219
      0.          -513.83740234  -381.3157959   -398.30310059
   -555.9921875   -121.46380615]
 [ -357.71896362  -962.51599121 -1015.74230957  -680.15093994
   -666.33557129     0.          -534.38995361  -856.23693848
   -422.42626953  -573.88012695]
 [ -819.02593994 -1104.11669922  -689.6842041  -1209.65405273
   -659.09802246  -546.53356934     0.         -1378.28796387
   -805.45489502 -1012.93432617]
 [ -484.87252808  -614.40661621  -4

In [37]:
sagemaker_session.delete_endpoint(endpoint_name=predictor.endpoint_name)


INFO:sagemaker:Deleting endpoint with name: pytorch-training-2023-10-27-19-49-51-655


ClientError: An error occurred (ValidationException) when calling the DeleteEndpoint operation: Could not find endpoint "pytorch-training-2023-10-27-19-49-51-655".